# Lecture 6 - May 24, 2017

$$Q = \text{ SELECT DISTINT } e_1\text{ AS } i_1 \ldots$$
$$\text{FROM R p } | (Q)p$$
$$\text{WHERE } e_1 = e_2 | e \text{ (NOT) IN } Q | \text{ (NOT) EXISTS} Q$$
$$\text{| }Q_1 \text{ UNION } Q_2$$
$$\text{| }Q_1 \text{ EXISTS } Q_2$$

**Recall**: For UNION and EXISTS, need union compatibility.

### Slide 43

```SQL
SELECT apubid
FROM article a
WHERE a.endpage-a.startpage > ALL (
  SELECT b.endpage-b.startpage
  FROM article b
  WHERE a.pubid != b.pubid
)
```
$$\{p | \exists c, s, e. \text{ article}(p,c,s,e) \wedge \forall p',c's',e' \text{ article}(p',c',s',e') \implies p \neq p' \implies e-s > e'-s'\}$$


```SQL
SELECT pubid
FROM article
WHERE endpage - startpage >= ALL (
  SELECT endpage-startpage
  FROM article
)
```
$$\{p | \exists c, s, e. \text{ article}(p,c,s,e) \wedge \forall p',c's',e' \text{ article}(p',c',s',e') \implies e-s \ge e'-s'\}$$

Are the two queries the same? Yes

### Slide 49
$$\{n_1, n_2 | \exists a_1,a_2. \text{ author}(a_1,n_1) \wedge author(a_2,n_2) \wedge
(\forall p,t. \text{ publication}(p,t) \wedge \text{ wrote}(a_1,p) \implies wrote(a_2,p)\}$$

$$\neg\exists p,t \text{ publication}(p,t) \wedge \text{ wrote}(a_1,p) \wedge \neg \text{wrote}(a_2,p)$$

```SQL
SELECT a1.name, a2.name
FROM author a1, authora2
WHERE NOT EXISTS (
  SELECT *
  FROM publication p, wrote w1
  WHERE p.pubid = w1.publication
  AND a1.aid = w1.author
  AND NOT EXISTS (
    SELECT *
    FROM wrote w2
    WHERE p.pubid = w2.publication
    AND a2.aid = w2.author
  )
)
```

## Updating Tables

### Incremental Updates
Typically the tables are large, but the updates are small
1. Insertion of tuples (`INSERT`)
  - Constant tuple
  - results of queries
```SQL
    INSERT INTO r[(a1, ..., al)]
       VALUES (v1, ..., vk)
```

2. Deletion of tuples (`DELETE`)
  - Delete all tuples based on the conditions
  - To test conditional, use a `SELECT *`
```SQL
    DELETE FROM r
    WHERE cond
```
  - **Cursors**
    - Avaliable in embedded SQL
    - Only way to delete one out of two duplicate tuples
    
3. Modification of tuples (`UPDATE`)
  - Allows updating in-place
  - Items selected based on condtionals
  - Doesn't allow params from the `WHERE` statement, only from the relation.
```SQL
    UPDATE table
    set a='t'
    WHERE a, b, c
```

### Support for Transactions
Transactions allow for atomic operations, guaranteed that everything/nothing in the transaction will complete.
- Transaction starts with the first access of the database, until it sees `Commit`
- `Commit` makes the changes permanent
- Some implementations check the constraints after every insert, instead of on commit:
  - Pros: Makes implementation easier, less problems with concurrency
  - Cons: Two dependant tables, Employee.eid == Department.Manager, Employee.department == Department.did
    - Condition check would constantly fail
- `ROLLBACK`: Revert changes made in the transaction

### Aggregation
Aggregate column functions extend First-Order queries (i.e. cannot be expressed in relational calculus)
- Find number of tuples in a relation
- Sum values of an attribute (over the whole relation)
- Find min/max values of an attribute

```SQL
SELECT x1, ..., xk, agg1, ...,aggn
FROM Q
GROUP BY x1,...xk
```
- Any column that does not appear in an aggregate function, must appear in the `GROUP BY` clause
- For all selected columns $S$ and aggregate variables $A$, group clause must contain $S \setminus A$